# Import project dependecies

In [2]:
import math
import random
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import pandas_datareader as data_reader

from tqdm import tqdm_notebook, tqdm
from collections import deque

# Build the AI Trader Network

In [31]:
class AI_Trader():
    def __init__(self,state_size, action_space=3, model_name="AITrader"):
        
        self.state_size=state_size
        self.action_space=action_space
        self.memory=deque(maxlen=2000)
        self.inventory=[]
        self.model_name=model_name
        
        self.gamma=0.95
        self.epsilon=1.0
        self.epsilon_final=0.01
        self.epsilon_decay=0.995
        
        self.model=self.model_builder()
        
    def model_builder(self):
        
        model=tf.keras.models.Sequential()
        model.add(tf.keras.layers.Dense(units=32, activation='relu', input_dim=self.state_size))
        model.add(tf.keras.layers. Dense(units=64, activation="relu"))
        model.add(tf.keras.layers. Dense(units=128, activation="relu"))
        model.add(tf.keras.layers. Dense(units=self.action_space, activation="linear"))
        model.compile(loss="mse", optimizer=tf.keras.optimizers.Adam(lr=0.001))
        return model
    
    def trade(self, state):
        if random.random()<=self.epsilon:
            return random.randrange(self.action_space)
        
        actions=self.model.predict(state)
        return np.argmax(actions[0])
    
    def batch_train(self, batch_size):
        
        batch=[]
        for i in range(len(self.memory) - batch_size + 1 , len(self.memory)):
            batch.append(self.memory[i])
            
        for state, action, reward, next_state, done in batch:
            reward=reward
            if not done:
                reward=reward+self.gamma * np.amax(self.model.predict(next_state)[0])
                
            target=self.model.predict(state)
            target[0][action]=reward
            
            self.model.fit(state, target, epochs=1, verbose=0)
        
        if self.epsilon>self.epsilon_final:
            self.epsilon*= self.epsilon_decay
        
        

# Dataset preprocessing

### Defining helper functions

In [32]:
# Sigmid

def sigmoid(x):
    return 1/(1+math.exp(-x))

In [33]:
# price format function

def stocks_price_format(n):
    if n<0:
        return "- $ {0:2f}".format(abs(n))
    else:
        return "$ {0:2f}".format(abs(n))

### Dataset Loader

In [34]:
def dataset_loader(stock_name):
  
    #Complete the dataset loader function
    dataset = data_reader.DataReader(stock_name, data_source="yahoo")
  
    start_date = str(dataset.index[0]).split()[0]
    end_date = str(dataset.index[-1]).split()[0]
  
    close = dataset['Close']

    return close

### State Creator

In [35]:
def state_creator(data, timestep, window_size):
    
    starting_id=timestep-window_size+1
    
    if starting_id>=0:
        windowed_data=data[starting_id:timestep+1]
    else:
        windowed_data=-starting_id*[data[0]]+list(data[0:timestep+1])
    
    state=[]
    for i in range(window_size-1):
        state.append(sigmoid(windowed_data[i+1]- windowed_data[i]))
        
    return np.array([state])
        

### Loading the dataset

In [36]:
stock_name="AAPL"
data=dataset_loader(stock_name)

# Training the AI Trader

### Setting hyper params

In [37]:
window_size=10
episodes=1000

batch_size=32
data_samples=len(data)-1

### Defining trader model

In [38]:
trader=AI_Trader(window_size)

In [39]:
trader.model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_8 (Dense)              (None, 32)                352       
_________________________________________________________________
dense_9 (Dense)              (None, 64)                2112      
_________________________________________________________________
dense_10 (Dense)             (None, 128)               8320      
_________________________________________________________________
dense_11 (Dense)             (None, 3)                 387       
Total params: 11,171
Trainable params: 11,171
Non-trainable params: 0
_________________________________________________________________


# Training loop

In [ ]:
for episode in range(1, episodes + 1):
  
    print("Episode: {}/{}".format(episode, episodes))
  
    state = state_creator(data, 0, window_size + 1)
  
    total_profit = 0
    trader.inventory = []
  
    for t in tqdm(range(data_samples)):
    
        action = trader.trade(state)
    
        next_state = state_creator(data, t+1, window_size + 1)
        reward = 0
    
        if action == 1: #Buying
            trader.inventory.append(data[t])
            print("AI Trader bought: ", stocks_price_format(data[t]))
      
        elif action == 2 and len(trader.inventory) > 0: #Selling
            buy_price = trader.inventory.pop(0)
      
            reward = max(data[t] - buy_price, 0)
            total_profit += data[t] - buy_price
            print("AI Trader sold: ", stocks_price_format(data[t]), " Profit: " + stocks_price_format(data[t] - buy_price) )
      
        if t == data_samples - 1:
            done = True
        else:
            done = False
      
        trader.memory.append((state, action, reward, next_state, done))
    
        state = next_state
    
        if done:
            print("########################")
            print("TOTAL PROFIT: {}".format(total_profit))
            print("########################")
    
        if len(trader.memory) > batch_size:
            trader.batch_train(batch_size)
      
    if episode % 10 == 0:
        trader.model.save("ai_trader_{}.h5".format(episode))
    

  0%|          | 0/1258 [00:00<?, ?it/s]

Episode: 1/1000
AI Trader bought:  $ 24.990000
AI Trader bought:  $ 24.879999
AI Trader sold:  $ 24.165001  Profit: - $ 0.824999
AI Trader sold:  $ 24.197500  Profit: - $ 0.682499
AI Trader bought:  $ 24.075001
AI Trader sold:  $ 25.355000  Profit: $ 1.279999
AI Trader bought:  $ 24.997499
AI Trader sold:  $ 23.522499  Profit: - $ 1.475000
AI Trader bought:  $ 23.620001
AI Trader bought:  $ 24.087500
AI Trader bought:  $ 24.150000
AI Trader bought:  $ 23.747499
AI Trader bought:  $ 23.567499
AI Trader bought:  $ 23.424999
AI Trader bought:  $ 24.160000
AI Trader sold:  $ 24.065001  Profit: $ 0.445000
AI Trader sold:  $ 24.010000  Profit: - $ 0.077499
AI Trader bought:  $ 24.219999
AI Trader bought:  $ 23.672501
AI Trader sold:  $ 24.025000  Profit: - $ 0.125000
AI Trader sold:  $ 24.190001  Profit: $ 0.442501
AI Trader bought:  $ 24.227501


  3%|▎         | 34/1258 [00:06<04:11,  4.88it/s]

AI Trader sold:  $ 25.132500  Profit: $ 1.565001


  3%|▎         | 35/1258 [00:08<07:02,  2.89it/s]

...

 86%|████████▋ | 1087/1258 [43:48<08:13,  2.88s/it]

AI Trader sold:  $ 75.157501  Profit: $ 2.889999


 86%|████████▋ | 1088/1258 [43:52<08:49,  3.12s/it]

AI Trader sold:  $ 75.934998  Profit: $ 2.644997


 87%|████████▋ | 1098/1258 [44:26<08:54,  3.34s/it]